## Event Detection

This notebooks shows an example of event detection on an MD LJ system.
The notebook will go through the data processing and detection pipeline.

### Import All the Things

In [ ]:
import bottleneck as bn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ruptures as rpt  # change point detection library
import scipy as sp
import scipy.stats
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler

import dupin as ed

FILENAME = "lj-data.h5"

### Import Trajectory Data

We also normalize all features to a range of $[0, 1]$.

In [ ]:
df = pd.read_hdf(FILENAME, "data")
df = pd.DataFrame(
    MinMaxScaler().fit_transform(df.to_numpy()),
    columns=df.columns,
    index=df.index)
df.head()

## Feature Selection

In order to lower the dimension of the data for interpretability and
improved detection, we will apply some feature selection methods to
the data.

### Mean Shift

First, we remove features where there is no detectable shift in the
mean between the start and end of the trajectory.

In [ ]:
mean_shift_filter = ed.preprocessing.filter.MeanShift(sensitivity=1e-4, )
data = df.to_numpy()
print(f"Starting number of features: {data.shape[1]}")
data = mean_shift_filter(data, sample_size=6)
print(f"New number of features: {data.shape[1]}")

### Correlated

We now take all features with a mean shift and cluster then
based on their correlation with other features. We will then
select 2 features for each cluster. The 2 features are chosen
based on a provided or random feature importance. In the next
cell we use 3 measures of feature importance:
1. The noise of the signal through a rolling standard deviation
   and mean.
2. The likelihoods of the observed mean shift from the `MeanShift`
   instance above.
3. The degree to which the feature is approximated by a linear
   spline with knots sparsely spaced in the signal.

The following cell clusters and selects features

In [ ]:
def weighted_average(*arrs, weights=None):
    avg_array = np.vstack(arrs)
    if weights is None:
        weights = np.ones(len(arrs)) / len(arrs)
    else:
        weights = np.asarray(weights)
    return np.sum(weights[:, None] * avg_array, axis=0)
    
# Rank based on local noise of signal
noise_importance = ed.preprocessing.filter.noise_importance(
    signal=data, window_size=4)

# Rank based on mean shift analysis
likelihoods = mean_shift_filter.likelihoods_[
    mean_shift_filter.filter_]
likelihood_importance = ed.preprocessing.filter.mean_shift_importance(
    likelihoods)

# Rank based on function smoothness
smoothness_importance = ed.preprocessing.filter.local_smoothness_importance(
    signal=data, dim=1, spacing=3)

importances = [
    noise_importance,
    likelihood_importance,
    smoothness_importance,
]
    
feature_score = weighted_average(*importances, weights=(0.25, 0.25, 0.5))

In [ ]:
correlation_filter = ed.preprocessing.filter.Correlated(max_clusters=6)

correlation_filter(
    data,
    features_per_cluster=2,
    feature_importance=feature_score,
    return_filter=True,
)

final_filter = np.flatnonzero(mean_shift_filter.filter_)[correlation_filter.filter_]

pruned_df = pd.DataFrame(
    df.iloc[:, final_filter],
    index=df.index,
    columns=df.columns[final_filter]
)
print(f"Number of clusters: {correlation_filter.n_clusters_}")
pruned_df.head()

## Change Point Detection

Now using the remaining feature we will attempt to detect any events in the
trajectory. To start will we check up to 12 change points and find the elbow
in the cost for entire signal given the $n$ change points using the KNEEDLE
algorithm. For the cost function we will use the built in `CostLinearFit`
which fits each dimension versus time to a line. We use the
`ruptures.Dynp` dynamic programming global optimizer for detection.

In [ ]:
lin_regress_cost = ed.detect.offline.CostLinearFit()
dynp = rpt.Dynp(custom_cost=lin_regress_cost)
sweep_detector = ed.detect.offline.SweepDetector(dynp, max_change_points=12)
change_points = sweep_detector.fit(pruned_df.to_numpy())
print(f"Optimal change points: {change_points}")

### Plot cost versus number of change points

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(sweep_detector.costs_)
min_ = min(sweep_detector.costs_)
max_ = max(sweep_detector.costs_)
ax.vlines(sweep_detector.opt_n_change_points_, min_, max_, "k", "--")

### Plot all features

In [ ]:
end_with_strs = ["2}$", "4}$", "6}$", "8}$", "10}$", "{vor}$"]
fig, axes = plt.subplots(3, 2, figsize=(10, 10))
for ax, end_str in zip(axes.ravel(), end_with_strs):
    filtered_df = df.filter(
        list(filter(lambda x: x.endswith(end_str), df.columns)), axis="columns"
    )
    max_, min_ = max(filtered_df.max()), min(filtered_df.min())
    filtered_df.plot(
        ax=ax, legend=False, title="_".join(filtered_df.columns[0].split("_")[-2:])
    )
    ax.title.set_size(20)
    ax.vlines(sweep_detector.opt_change_points_, max_, min_, linestyles="--")

### Plot only used features

In [ ]:
end_with_strs = ["2}$", "4}$", "6}$", "8}$", "10}$", "{vor}$"]
n_axes = 0
for end_str in end_with_strs:
    filtered_df = pruned_df.filter(
        list(filter(lambda x: x.endswith(end_str), pruned_df.columns)), axis="columns"
    )
    if filtered_df.shape[1] != 0:
        n_axes += 1

rows = int(np.ceil(n_axes / 2))
fig, axes = plt.subplots(rows, 2, figsize=(10, 4 * rows))
axes = iter(axes.ravel())
ax = next(axes)
for end_str in end_with_strs:
    filtered_df = pruned_df.filter(
        list(filter(lambda x: x.endswith(end_str), pruned_df.columns)), axis="columns"
    )
    if filtered_df.shape[1] != 0:
        filtered_df.plot(
            ax=ax, legend=False, title="_".join(filtered_df.columns[0].split("_")[-2:])
        )
        ax.title.set_size(20)
        max_, min_ = filtered_df.max().max(), filtered_df.min().min()
        ax.vlines(sweep_detector.opt_change_points_, max_, min_, linestyles="--")
        try:
            ax = next(axes)
        except StopIteration:
            pass
fig.tight_layout()

In [ ]:
pruned_df.to_hdf("lj-data.h5", "pruned")